In [66]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 24 18:32:18 2020

"""

#The basics
import pandas as pd
import numpy as np
import json

#Plotting
import matplotlib.pyplot as plt

#Statistical fitting of models
import statsmodels.api as sm
import statsmodels.formula.api as smf

import warnings 

#Decide which league to load
#Wyscout data from https://figshare.com/collections/Soccer_match_event_dataset/4415000/2
with open('../data/events_England.json') as f:
    data = json.load(f)
    
#Create a data set of shots.
train = pd.DataFrame(data)
pd.unique(train['subEventName'])
shots=train[train['subEventName']=='Shot']
shots_model=pd.DataFrame(columns=['Goal','X','Y'])
shots_model

for i,shot in shots.iterrows():
    
    header=0
    for shottags in shot['tags']:
        if shottags['id']==403:
            header=1
    #Only include non-headers        
    if not(header):        
        shots_model.at[i,'X']=100-shot['positions'][0]['x']
        shots_model.at[i,'Y']=shot['positions'][0]['y']
        shots_model.at[i,'C']=abs(shot['positions'][0]['y']-50)
    
        #Distance in metres and shot angle in radians.
        x=shots_model.at[i,'X']*105/100
        y=shots_model.at[i,'C']*65/100
        shots_model.at[i,'Distance']=np.sqrt(x**2 + y**2)
        a = np.arctan(7.32 *x /(x**2 + y**2 - (7.32/2)**2))
        if a<0:
            a=np.pi+a
        shots_model.at[i,'Angle'] =a
    
        #Was it a goal
        shots_model.at[i,'Goal']=0
        for shottags in shot['tags']:
                #Tags contain that its a goal
                if shottags['id']==101:
                    shots_model.at[i,'Goal']=1

model_variables = ['Angle','Distance']
model=''
for v in model_variables[:-1]:
    model = model  + v + ' + '
model = model + model_variables[-1]

#Fit the model
test_model = smf.glm(formula="Goal ~ " + model, data=shots_model, 
                           family=sm.families.Binomial()).fit()      
b=test_model.params


def calculate_xG(sh):    
   bsum=b[0]
   for i,v in enumerate(model_variables):
       bsum=bsum+b[i+1]*sh[v]
   xG = 1/(1+np.exp(bsum)) 
   return xG   

#Create a 2D map of xG
x =80
y=32.5
sh=dict()
a = np.arctan(7.32 *x /(x**2 + abs(y-65/2)**2 - (7.32/2)**2))
if a<0:
    a = np.pi + a
sh['Angle'] = a
print(a)
sh['Distance'] = np.sqrt(x**2 + abs(y-65/2)**2)
print(np.sqrt(x**2 + abs(y-65/2)**2))
sh['D2'] = x**2 + abs(y-65/2)**2
sh['X'] = x
sh['AX'] = x*a
sh['X2'] = x**2
sh['C'] = abs(y-65/2)
sh['C2'] = (y-65/2)**2
        
final =  calculate_xG(sh)
final = ("%.17f" % final).rstrip('0').rstrip('.')
print(final)

0.09143624164475057
80.0
0.00005181788913153
